# **1CK130: Assignment 4**

*Lecturers: dr. Bram De Moor, dr. Mirjam Meijer, Emre Kara Msc.*

---

Velocation, the bike-rental operator based in Paris, was very satisfied with the work you completed in the previous assignments. As a result, they have once again requested your assistance to help optimize their logistics network of 48 bike rental stations. Unlike the earlier assignments, this one introduces a multi-period setting: the distribution of bikes across all rental stations at the end of each day becomes the starting distribution for the following day. By designing an efficient nightly rebalancing strategy, you will aim to optimize the overall performance of Velocation’s bike-sharing network. To support this analysis, you are provided with data, described below, that is similar to Assignment 3.


## Data

Like in Assignment 3, you have access to the following data: `bike_locations_data.xlsx` and `transactions_data.xlsx`. `bike_locations_data.xlsx` contains the location in x- and y-coordinates of 48 bike rental stations. `transactions_data.xlsx` contains data on bike withdrawals and returns for six representative days. Per transaction (a row in `transactions_data.xlsx`), the following data is provided in columns:
* `oper_day`: an id (between 0 and 5) that connects to the day for which this data was collected. The data contains all trips that were started between 00:00 and 23:59 on the respective days.
* `wdr_loc`: location where the bike was picked up.
* `wdr_b`: number of bikes at the withdrawal location, just after this bike was withdrawn.
* `wdr_t`: time at which the bike was withdrawn (number representing the amount of hours after 00:00. I.e. 6.5 refers to six and a half hours after midnight, i.e. 06:30AM.)
* `ret_loc`: location where the bike was returned.
* `ret_b`: number of bikes at the return location, just after this bike was returned.
* `ret_t`: time at which bike was returned (same encoding as `wdr_t`).

You are encouraged to build upon the data analysis performed in Assignment 3 to conduct the analyses for this assignment. To load the data, you need to correctly mount the drive, similar to Assignment 3:

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


After successfully mounting the drive, the code below loads the data as dataframes. Note that you may need to change `file_path` depending on where in Google Drive you stored the excel file.

In [ ]:
import pandas as pd
file_path = '/content/drive/My Drive/1CK130/2025/Assignments/Data/'
locations_data = pd.read_excel(file_path + 'bike_locations_data.xlsx')
transactions_data = pd.read_excel(file_path + 'transactions_data.xlsx')

## PART I: Evaluating the network's imbalance

Before optimizing Velocation’s rebalancing operations, it is essential to first determine whether rebalancing is actually needed. To do so, analyze how the distribution of bikes across stations evolves over time when there is no nightly rebalancing and compute the average service level of the network, defined as: 1 - (customers turned away / total customers).
Assume there is a total of 1248 bikes, initially distributed equally among all stations (so each station has 26 bikes). Ensure that your analysis is statistically rigorous and supported by clear, insightful visualizations. You are encouraged to use and adapt code from the previous assignments.

In [ ]:
# ADD YOUR CODE HERE.

## PART II: Optimizing rebalancing operations

Having established that some degree of rebalancing may be necessary, your next task is to design efficient strategies to redistribute the 1248 bikes during the night. To gradually build toward your final solution, address the following questions:

1. What would be the ideal starting distribution of bikes at the beginning of the day to maximize the service level? In other words, if there were no constraints on the rebalancing operations, how would you allocate the bikes accross the stations?
2. In practice, the number of bikes that can be moved overnight is limited. Assume that Velocation operates vans, each with a capacity of 30 bikes per night, regardless of the number of pick-up and drop-off locations. How much can the service level be improved by adding one, two, three, or more vans?
3. Using vans is costly, as are multiple stops for pick-up and drop-off operations. For Velocation, the fixed cost to deploy a van is €50 per night. Additionally, each stop at a station (for either pick-up or drop-off) incurs an extra cost of €10. Each van can move at most 30 bikes per night. Your task is to design a rebalancing strategy that achieves an average service level of at least 95% while minimizing the average operational cost per night. Implement this strategy as a function that takes the end-of-day bike distribution (formatted as a list) as input and outputs a matrix indicating, for each element `[i,j]`, the number of bikes transferred from station *i* to station *j* during the night. Use clear visuals to present and interpret your final results.

In [ ]:
# ADD YOUR CODE HERE.

## Grading

When grading, we consider the following:

1.   Does your code run? Is it clear and well-annotated? (10%)
2.   Are your assumptions motivated and reasonable? (20%)
3. How innovative is your optimization approach and how well does it perform? (20%)
4. Do you present your results in a clear, statistically correct, and visually appealing manner? (20%)
5.   Do you correctly interpret and discuss your results? (15%)
6.   Do you manage to present your simulation model well and are able to answer questions related to it (during the presentation)? (15%)
